#### 의존성 설치

In [ ]:
%pip install -r requirements.txt

#### MongoDB 연결

In [ ]:
# Connection Info (환경 변수에서 값 불러오기)
from dotenv import load_dotenv
import os

load_dotenv()

USERNAME = os.getenv("MONGO_USERNAME")
PASSWORD = os.getenv("MONGO_PASSWORD")
HOST = os.getenv("MONGO_HOST")
PORT = int(os.getenv("MONGO_PORT"))

In [ ]:
url = f"mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/"

In [ ]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    client = MongoClient(url)
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")

except ConnectionFailure as e:
    print(f"MongoDB connection failed: {e}")

#### DB와 Collection 생성

In [ ]:
db = client['s307_db']
collection = db['s307_collection']

In [ ]:
print(collection)

#### pdf 파일 선정

In [ ]:
sample_pdf = "gpt-020-3m-sds.pdf"

#### 이미지 데이터 전처리 함수
stream 객체 제거, colorspace 객체 전처리
filter, colorspace 등 이미지 로더 조건 분기(아직 미흡)

In [ ]:
# 원본 코드
from PIL import Image
from io import BytesIO

# 직렬화 되지 않는 이미지 정보 처리 + 이미지 저장 + url 추가
def make_image_doc_serializable(page_num: int, img: dict) -> dict:
    # 이미지 폴더 생성 (없으면 자동 생성)
    os.makedirs("images", exist_ok=True)

    # png 제작을 위한 이미지 정보 추출
    img_id = str(img['name'])
    img_data = img['stream'].get_data()    # 바이트
    img_info = img['stream'].attrs
    img_width = int(img_info['Width'])
    img_height = int(img_info['Height'])
    img_bits = int(img_info['BitsPerComponent'])

    img_colorspace = str(img_info['ColorSpace']).lstrip('/').upper()
    print(img_colorspace[:10])
    img_filter_raw = img_info.get('Filter')
    img_filters = (
        [] if img_filter_raw is None
        else ([str(x).lstrip('/').strip(" '\"").upper() for x in img_filter_raw]
            if isinstance(img_filter_raw, (list, tuple))
            else [str(img_filter_raw).lstrip('/').strip(" '\"").upper()])
    )
    has_jpeg_like = any(f in ('DCTDECODE', 'JPXDECODE') for f in img_filters)

    # 이미지 모드 설정
    if img_colorspace in ('RGB', 'DEVICERGB'):
        img_mode = 'RGB'
    elif img_colorspace in ('GRAY', 'DEVICEGRAY'):
        img_mode = '1' if img_bits == 1 else 'L'
    elif img_colorspace in ('CMYK', 'DEVICECMYK'):
        img_mode = 'CMYK'
    else:
        img_mode = 'RGB'
        
    img_size = (img_width, img_height)



    # filter별 이미지 처리
    if has_jpeg_like:
        print("jpeg 이미지 발견\n")
        try:
            pil_image = Image.open(BytesIO(img_data))
            pil_image = pil_image.convert('RGB')
            pil_image.save(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            img_url = str(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            print(f"{img_id} 이미지 저장 완료\n")
        except Exception as e:
            print(f"이미지 저장 오류: {e}")
            img_url = None
    else:
        print("그 외 이미지 발견\n")
        try:
            pil_image = Image.frombytes(img_mode, img_size, img_data)
            pil_image = pil_image.convert('RGB')
            pil_image.save(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            img_url = str(f"images/{sample_pdf}_{page_num}_{img_id}.png")
            print(f"{img_id} 이미지 저장 완료\n")
        except Exception as e:
            print(f"이미지 저장 오류: {e}")
            img_url = None
        
    
    out = {k: v for k, v in img.items() if k != "stream"}  # stream 제거
    if "colorspace" in out:
        out["colorspace"] = [str(cs).lstrip("/") for cs in out["colorspace"]]
    if isinstance(out.get("srcsize"), tuple):
        out["srcsize"] = list(out["srcsize"])

    out['url'] = img_url
    return out

In [ ]:
import pdfplumber

pdf =  pdfplumber.open(sample_pdf)
page_count = len(pdf.pages)

# Root 객체 생성
root = {
    "file_name": sample_pdf,
    "page_num": 0,
    "page_count": page_count
}

# Root 객체 추가
root_insert = collection.insert_one(root)
print(f"Inserted Root | File: {root['file_name']} | Page Count: {root['page_count']} | ID: {root_insert.inserted_id}")

for i in range(page_count):
    page = pdf.pages[i]
    page_num = int(page.page_number)
    keys = list(page.objects.keys())

    # # # 이미지 데이터 전처리
    if "image" in keys and page.images:
        images_wo_stream = [make_image_doc_serializable(page_num, img) for img in page.images]


    # # 키에 존재하는 값만 추가
    page_objects = {
        "file_name": sample_pdf,
        "page_num": page.page_number,
        "keys": list(page.objects.keys()),
        **({"lines": page.lines} if "line" in keys else {}),
        **({"rects": page.rects} if "rect" in keys else {}),
        **({"chars": page.chars} if "char" in keys else {}),
        **({"curves": page.curves} if "curve" in keys else {}),
        **({"images": images_wo_stream} if "image" in keys else {}),
    }


    result = collection.insert_one(page_objects)
    print(f"Inserted File: {sample_pdf} | Page: {page_num} | ID: {result.inserted_id}")

pdf.close()

#### DB 연결 종료

In [ ]:
client.close()

#### 데이터 확인 및 시각화

In [ ]:
# 다시 MongoDB 연결

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from dotenv import load_dotenv
import pdfplumber
import os

load_dotenv()

USERNAME = os.getenv("MONGO_USERNAME")
PASSWORD = os.getenv("MONGO_PASSWORD")
HOST = os.getenv("MONGO_HOST")
PORT = int(os.getenv("MONGO_PORT"))

url = f"mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/"


try:
    client = MongoClient(url)
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")

except ConnectionFailure as e:
    print(f"MongoDB connection failed: {e}")

db = client['s307_db']
collection = db['s307_collection']

In [ ]:
# 한 페이지의 전체 구조 확인
# doc = collection.find_one({"file_name": sample_pdf, "page_num": 1})
# doc['chars'][0].keys()

In [ ]:
# image 타입 데이터 확인
# print("\n=== image 타입 데이터 ===")
# if doc['images'] and len(doc['images']) > 0:
#     print(f"image 개수: {len(doc['images'])}")
#     print("\n첫 번째 image 객체:")
#     first_image = doc['images'][0]
#     print(f"전체 데이터: {first_image}")
    
# else:
#     print("image 데이터가 없습니다.")

In [ ]:
# 테스트용 - 기존 데이터 정리
# print("=== 기존 데이터 정리 ===")
# result = collection.delete_many({"file_name": sample_pdf})
# print(f"삭제된 문서 수: {result.deleted_count}")

In [ ]:
# # 시각화 전 좌표 변환 함수 설정
def convert_pdf_to_display_coords(bbox, page_height):
    """PDF 좌표를 시각화용 좌표로 변환"""
    x0, y0, x1, y1 = bbox
    # Y축 뒤집기: PDF 좌표계 -> Pillow 좌표계
    new_y0 = page_height - y1
    new_y1 = page_height - y0
    return [x0, new_y0, x1, new_y1]

In [ ]:
# 확인할 파일 이름

# 조회할 페이지 번호
target_page_num = 1

# pdf에서 이미지 생성
pdf = pdfplumber.open(sample_pdf)
page = pdf.pages[target_page_num - 1]
page_height = page.height

# MongoDB에서 조회한 데이터
page_data = collection.find_one({"file_name": sample_pdf, "page_num": target_page_num})

if page_data and "keys" in page_data:
    print(f"총 {len(page_data.get('keys', []))}개의 타입을 찾았습니다.")
    
    # 시각화
    img = page.to_image()
    
    # 타입별로 분리해서 그리기
    images = page_data.get("images", [])
    chars = page_data.get("chars", [])
    lines = page_data.get("lines", [])
    rects = page_data.get("rects", [])
    
    print(f"이미지: {len(images)}개, 문자: {len(chars)}개, 라인: {len(lines)}개, 사각형: {len(rects)}개")
    
    # 각 타입별로 사각형 그리기 (좌표 변환 적용)
    if images:
        img_rects = []
        for obj in images:
            # PDF 좌표를 시각화용 좌표로 변환
            # 빨간색 표시
            bbox = [obj["x0"], obj["y0"], obj["x1"], obj["y1"]]
            converted_bbox = convert_pdf_to_display_coords(bbox, page_height)
            img_rects.append({
                "x0": converted_bbox[0],
                "y0": converted_bbox[1], 
                "x1": converted_bbox[2],
                "y1": converted_bbox[3],
                "top": converted_bbox[1],
                "bottom": converted_bbox[3]
            })
        img.draw_rects(img_rects, fill=(255, 0, 0, 50))
        print(f"이미지 {len(images)}개 그리기 완료")
    
    if chars:
        char_rects = []
        for obj in chars:
            # 파란색 표시
            bbox = [obj["x0"], obj["y0"], obj["x1"], obj["y1"]]
            converted_bbox = convert_pdf_to_display_coords(bbox, page_height)
            char_rects.append({
                "x0": converted_bbox[0],
                "y0": converted_bbox[1], 
                "x1": converted_bbox[2],
                "y1": converted_bbox[3],
                "top": converted_bbox[1],
                "bottom": converted_bbox[3]
            })
        img.draw_rects(char_rects, fill=(0, 0, 255, 50))
        print(f"문자 {len(chars)}개 그리기 완료")
    
    if lines:
        line_rects = []
        for obj in lines:
            # 노란색 선 표시
            bbox = [obj["x0"], obj["y0"], obj["x1"], obj["y1"]]
            converted_bbox = convert_pdf_to_display_coords(bbox, page_height)
            line_rects.append({
                "x0": converted_bbox[0],
                "y0": converted_bbox[1], 
                "x1": converted_bbox[2],
                "y1": converted_bbox[3],
                "top": converted_bbox[1],
                "bottom": converted_bbox[3]
            })
        img.draw_rects(line_rects, fill=(255, 255, 0, 100), stroke=(0, 0, 0, 255))
        print(f"라인 {len(lines)}개 그리기 완료")
    
    if rects:
        rect_rects = []
        for obj in rects:
            # 자홍색 표시, 주황 선 표시 #fill=(255, 0, 255, 150)
            bbox = [obj["x0"], obj["y0"], obj["x1"], obj["y1"]]
            converted_bbox = convert_pdf_to_display_coords(bbox, page_height)
            rect_rects.append({
                "x0": converted_bbox[0],
                "y0": converted_bbox[1], 
                "x1": converted_bbox[2],
                "y1": converted_bbox[3],
                "top": converted_bbox[1],
                "bottom": converted_bbox[3]
            })
        img.draw_rects(rect_rects,fill=(255, 0, 255, 150),stroke=(255, 165, 0, 255))
        print(f"사각형 {len(rects)}개 그리기 완료")
    
    img.show()
    pdf.close()
else:
    print("데이터를 찾을 수 없습니다.")